In [1]:
import pandas as pd
import re

In [2]:
events_path = "../data/events.csv"
events = pd.read_csv(events_path)

### Parse Promotion Name

In [3]:
for index_label, row_series in events.iterrows():
    event_name = row_series["NAME"]
    promotion_name = event_name.split(" -",1)[0]
    
    # Check if last character is number
    if promotion_name[-1].isdigit():
        promotion_name = re.compile(r" [0-9]+$").split(promotion_name)[0]
        
    if promotion_name == "UFC Fight Night":
        promotion_name = "UFC"
        
    if promotion_name == "UFC on Fox":
        promotion_name = "UFC"
        
    if promotion_name == "PLMMA 61 / AFC":
        promotion_name = "PLMMA"
        
    if promotion_name == "UFC on Fuel TV":
        promotion_name = "UFC"
        
    if promotion_name == "UFC on ESPN":
        promotion_name = "UFC"
        
    if promotion_name == "UFC Live":
        promotion_name = "UFC"
    
    events.at[index_label , 'PROMOTION'] = promotion_name

In [4]:
events.head(10)

,NAME,DATE,PROMOTION
0,UFC 228 - Woodley vs. Till,Sep / 08 / 2018,UFC
1,UFC 220 - Miocic vs. Ngannou,Jan / 20 / 2018,UFC
2,UFC 218 - Holloway vs. Aldo 2,Dec / 02 / 2017,UFC
3,UFC Fight Night 109 - Gustafsson vs. Teixeira,May / 28 / 2017,UFC
4,UFC Fight Night 99 - Mousasi vs. Hall 2,Nov / 19 / 2016,UFC
5,LFC 61 - Legacy Fighting Championship 61,Oct / 14 / 2016,LFC
6,Rage in the Cage OKC 47 - Night of Champions,Aug / 13 / 2016,Rage in the Cage OKC
7,Bellator 143 - Warren vs. Davis,Sep / 25 / 2015,Bellator
8,RXMMA - Rocks Xtreme MMA 12,Feb / 28 / 2015,RXMMA
9,Bellator MMA - Bellator 111,Mar / 07 / 2014,Bellator MMA


### Join Events to Fights table

In [5]:
fights_path = "../data/fights.csv"
fights = pd.read_csv(fights_path)

In [6]:
events_fights = pd.merge(events, fights, left_on='NAME', right_on='Event')
events_fights.head()

,NAME,DATE,PROMOTION,Event,Win/Loss/Draw,Fighter,Opponent,Method,Referee,Round,Time,Closed Odds
0,UFC 228 - Woodley vs. Till,Sep / 08 / 2018,UFC,UFC 228 - Woodley vs. Till,WIN,ABDUL RAZAK ALHASSAN,NIKO PRICE,KO (Punch),Jacob Montalvo,1,0:43,-130.0
1,UFC 228 - Woodley vs. Till,Sep / 08 / 2018,UFC,UFC 228 - Woodley vs. Till,LOSS,ALEX WHITE,JIM MILLER,Submission (Rear-Naked Choke),Kevin MacDonald,1,1:29,-185.0
2,UFC 228 - Woodley vs. Till,Sep / 08 / 2018,UFC,UFC 228 - Woodley vs. Till,WIN,ALJAMAIN STERLING,CODY STAMANN,Submission (Kneebar),Kevin MacDonald,2,3:42,-190.0
3,UFC 228 - Woodley vs. Till,Sep / 08 / 2018,UFC,UFC 228 - Woodley vs. Till,LOSS,BRANDON DAVIS,ZABIT MAGOMEDSHARIPOV,Submission (Kneebar),Jacob Montalvo,2,4:36,610.0
4,UFC 228 - Woodley vs. Till,Sep / 08 / 2018,UFC,UFC 228 - Woodley vs. Till,LOSS,CARLA ESPARZA,TATIANA SUAREZ,TKO (Punches),Kerry Hatley,3,4:33,300.0


### Loop through fighters and Add up UFC wins

In [7]:
fighters_path = "../data/fighters.csv"
fighters = pd.read_csv(fighters_path)

for index_label, row_series in fighters.iterrows():
    fighter_name = fighters["NAME"].values[index_label]
    
    total_wins = len(events_fights[(events_fights.Fighter == fighter_name) & (events_fights["Win/Loss/Draw"] == "WIN")])
    total_losses = len(events_fights[(events_fights.Fighter == fighter_name) & (events_fights["Win/Loss/Draw"] == "LOSS")])

    ufc_wins = len(events_fights[(events_fights.Fighter == fighter_name) & (events_fights["Win/Loss/Draw"] == "WIN") & (events_fights['PROMOTION'] == "UFC")])
    ufc_losses = len(events_fights[(events_fights.Fighter == fighter_name) & (events_fights["Win/Loss/Draw"] == "LOSS") & (events_fights['PROMOTION'] == "UFC")])
    
    fighters.at[index_label , 'TOTAL_WINS'] = total_wins
    fighters.at[index_label , 'TOTAL_LOSSES'] = total_losses
    fighters.at[index_label , 'UFC_WINS'] = ufc_wins
    fighters.at[index_label , 'UFC_LOSSES'] = ufc_losses
    
fighters.head(10)

,NAME,GYM,DOB,HEIGHT,WEIGHT,TOTAL_WINS,TOTAL_LOSSES,UFC_WINS,UFC_LOSSES
0,ABDUL RAZAK ALHASSAN,NaN,1985-08-11,5'10',170 LBS,10.0,1.0,4.0,1.0
1,ABDUL-KERIM EDILOV,Akhmat Fight Team,1991-11-25,6'3',205 LBS,17.0,4.0,1.0,0.0
2,ABU AZAITAR,Team Azaitar,1986-03-10,5'9',185 LBS,14.0,1.0,1.0,0.0
3,ADAM MILSTEAD,Pittsburgh Fight Club,1987-07-03,6'3',205 LBS,9.0,3.0,1.0,2.0
4,ADAM WIECZOREK,AKS Chorzow,1992-02-05,6'5',232 LBS,10.0,2.0,2.0,1.0
5,ADAM YANDIEV,Fight or Die,1988-07-07,5'11',185 LBS,9.0,1.0,0.0,1.0
6,ADLAN AMAGOV,Jackson-Wink MMA,1986-10-30,5'11',170 LBS,15.0,3.0,2.0,0.0
7,AIEMANN ZAHABI,Tristar Gym,1987-11-19,5'8',135 LBS,7.0,2.0,1.0,2.0
8,AISLING DALY,SBG Ireland,1987-12-24,5'3',116 LBS,17.0,7.0,3.0,2.0
9,AL IAQUINTA,Serra-Longo Fight Team,1987-04-30,5'7',155 LBS,21.0,5.0,13.0,4.0


### Add up UFC wins

In [8]:
wins = len(events_fights[(events_fights.Fighter == "ALJAMAIN STERLING") & (events_fights["Win/Loss/Draw"] == "WIN")])
losses = len(events_fights[(events_fights.Fighter == "ALJAMAIN STERLING") & (events_fights["Win/Loss/Draw"] == "LOSS")])

ufc_wins = len(events_fights[(events_fights.Fighter == "ALJAMAIN STERLING") & (events_fights["Win/Loss/Draw"] == "WIN") & (events_fights['PROMOTION'] == "UFC")])
ufc_losses = len(events_fights[(events_fights.Fighter == "ALJAMAIN STERLING") & (events_fights["Win/Loss/Draw"] == "LOSS") & (events_fights['PROMOTION'] == "UFC")])

print("All wins and losses")
print(wins, losses)
print("UFC wins and losses")
print(ufc_wins, ufc_losses)

All wins and losses
20 4
UFC wins and losses
10 3
